In [1]:
from pyMSOO.MFEA.model import MFEA_base, SM_MFEA, LSA21
from pyMSOO.MFEA.competitionModel import SM_MFEA_Competition
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.utils.Search import * 
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 

from pyMSOO.utils.EA import * 
from pyMSOO.MFEA.benchmark.continous.CEC17 import CEC17_benchmark 
from pyMSOO.MFEA.benchmark.continous.WCCI22 import WCCI22_benchmark
from pyMSOO.MFEA.benchmark.continous.funcs import * 
from pyMSOO.utils.DimensionAwareStrategy import DaS_strategy, NoDaS
from pyMSOO.utils.MultiRun.RunMultiTime import * 
from pyMSOO.utils.MultiRun.RunMultiBenchmark import * 

from pyMSOO.utils.LoadSaveModel.load_utils import loadModel, loadModelFromTxt
from pyMSOO.utils.LoadSaveModel.save_utils import export_history2txt

from pyMSOO.utils.numba_utils import *
from pyMSOO.utils.Compare.compareModel import CompareModel

import pandas as pd 
import numpy as np 
import os 

In [3]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(1)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()

In [6]:
baseModel = SM_MFEA.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    crossover = DaS_SBX_Crossover(nc= 2, eta= 0.001, conf_thres= 1),
    # crossover = SBX_Crossover(nc = 2),
    mutation = PolynomialMutation(nm = 5, pm= 1),
    selection= ElitismSelection(random_percent= 0.1),
    search= L_SHADE(len_mem= 15),
    attr_tasks = ['crossover', 'mutation', 'search'],
)
solve = baseModel.fit(
    nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 20,
    lr = 0.1, p_const_intra= 0., prob_search = 0., lc_nums = 200,
    nb_epochs_stop= 1000, swap_po= False,
    evaluate_initial_skillFactor= True
)

Seed: None -- Time: 01m 21.58s  100 % [====================>]  Pop_size: 1.90E+02  ,  Cost: 1.31E-18  1.05E-16  0.00E+00  0.00E+00  1.34E+02  4.54E-10  0.00E+00  6.36E-04  0.00E+00  9.95E-01  ,  
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[100226, 100790, 100016, 100162, 99966, 100152, 99188, 99884, 99652, 99994]
END!


In [13]:
ls_benchmark = []
ls_IndClass = []
ls_tasks = [2, 3, 4, 5, 6, 7, 8, 9, 10]
name_benchmark = [] 

for i in ls_tasks:
    # t, ic = WCCI22_benchmark.get_complex_benchmark(i)
    t, ic = WCCI22_benchmark.get_50tasks_benchmark(i)
    ls_benchmark.append(t[:10])
    ls_IndClass.append(ic)
    name_benchmark.append(str(i))

smpModel = MultiBenchmark(
    ls_benchmark= ls_benchmark,
    name_benchmark= name_benchmark,
    ls_IndClass= ls_IndClass,
    model= SM_MFEA
)

smpModel.compile( 
        crossover= SBX_Crossover(nc = 2),
        mutation= PolynomialMutation(nm = 5, pm=1),
        dimension_strategy= NoDaS(),
        search = DifferentialEvolution.L_SHADE(p_ontop= 0.11, len_mem= 30),
        selection = ElitismSelection(random_percent= 0.0)
)
smpModel.fit(
        nb_generations= 200, nb_inds_each_task= 100, nb_inds_min= 20,
        lr = 0.1 ,mu= 0.1,
        evaluate_initial_skillFactor= True  
)
a = smpModel.run(
    nb_run=15,     
    save_path= './RESULTS/check/'
)

Seed: 0 -- Time: 00m 10.30s  100 % [====================>]  Pop_size: 1.90E+02  ,  Cost: 1.63E+03  7.46E+02  3.22E+02  2.07E+03  5.12E+02  1.50E+03  6.34E+02  1.81E+03  2.41E+03  1.66E+03  ,  
END!
Seed: 1 -- Time: 00m 10.27s  100 % [====================>]  Pop_size: 1.90E+02  ,  Cost: 2.39E+03  3.48E+02  3.14E+03  5.36E+02  8.15E+02  2.81E+03  5.06E+02  7.37E+02  6.50E+02  2.20E+03  ,  
END!
Seed: 2 -- Time: 00m 10.49s  100 % [====================>]  Pop_size: 1.90E+02  ,  Cost: 2.05E+02  2.54E+02  2.20E+02  1.97E+02  2.61E+02  8.96E+01  4.59E+02  5.15E+02  6.24E+02  1.58E+02  ,  
END!
Seed: 3 -- Time: 00m 10.97s  100 % [====================>]  Pop_size: 1.90E+02  ,  Cost: 3.80E+02  5.50E+02  4.33E+02  9.90E+02  5.78E+02  8.41E+02  8.31E+02  5.37E+02  6.28E+02  8.20E+02  ,  
END!
Seed: 4 -- Time: 00m 10.93s  100 % [====================>]  Pop_size: 1.90E+02  ,  Cost: 1.21E+02  2.82E+02  1.28E+02  1.62E+02  1.01E+03  5.10E+02  6.55E+01  6.68E+02  2.20E+02  2.04E+02  ,  
END!
Seed: 5 --

In [12]:
model = loadModel("./RESULTS/check/2.mso", ls_tasks= WCCI22_benchmark.get_50tasks_benchmark(2)[0])
model.history_cost[-1]

array([2091.48206157,  735.9514915 ,  336.582458  ,  564.95256945,
        908.10868506, 1084.22144928,  667.73439907, 1442.081112  ,
        597.56944777, 1071.37728011])

In [ ]:
model.